<a href="https://colab.research.google.com/github/sanderson0398/record-metadata/blob/main/FS_008.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Cleaning for "000-Leader" and "008-Fixed-Length Data"

In [8]:
import pandas as pd
import numpy as np
from tabulate import tabulate

# Step 1: Define file path and read CSV with error handling
csv_file_path = '/content/FS_Book_cat.csv'
try:
    df = pd.read_csv(csv_file_path, on_bad_lines='skip', low_memory=False)  # Handles bad lines gracefully
except pd.errors.ParserError as e:
    print(f"Error reading CSV: {e}")
    raise

# Step 2: Clean data by removing columns with only NA values
df_cleaned = df.dropna(axis=1, how='all')

# Step 3: Filter columns with specific prefixes
prefixes = [
    '000','001', '008', '245$6', '245$a', '245$b', '245$f', '245$n', '245$p',
    '260$a', '260$b', '260$c',
    '264$a', '264$b', '264$c', '773$w'
]
df_filtered = df_cleaned[[col for col in df_cleaned.columns if any(col.startswith(prefix) for prefix in prefixes)]]

# Step 4: Split '008-Fixed-Length Data Elements-General Information' into separate columns
def split_008_column(row):
    fixed_data = row['008-Fixed-Length Data Elements-General Information']
    return pd.Series({
        'Record Creation Date': fixed_data[:6] if len(fixed_data) >= 6 else np.nan,  # First 6 characters (YYMMDD)
        'Publication Status': fixed_data[6] if len(fixed_data) >= 7 else np.nan,  # Character 7
        'Date 1': fixed_data[7:11] if len(fixed_data) >= 11 else np.nan,  # Characters 8-11
        'Date 2': fixed_data[11:15] if len(fixed_data) >= 15 else np.nan,  # Characters 12-15
        'Place of Publication': fixed_data[15:18] if len(fixed_data) >= 18 else np.nan,  # Characters 16-18
        'Language': fixed_data[35:38] if len(fixed_data) >= 38 else np.nan,  # Characters 36-38
        'Modified Record': fixed_data[38] if len(fixed_data) >= 39 else np.nan  # Character 39
    })

if '008-Fixed-Length Data Elements-General Information' in df_filtered.columns:
    df_split = df_filtered.apply(split_008_column, axis=1)
    df_split = df_split.replace('|||||', np.nan)  # Replace placeholder with NaN
    df_combined = pd.concat([df_filtered, df_split], axis=1)
else:
    df_combined = df_filtered

# Step 5: Add 'Bibliography' column from '000-Leader'
df_combined['Bibliography'] = df_combined['000-Leader'].str[6]  # 7th character is at index 6
df_combined['6th'] = df_combined['000-Leader'].str[5]  # 6th character (index starts from 0)
df_combined['8th'] = df_combined['000-Leader'].str[7]  # 8th character



In [9]:
df_combined.columns

Index(['000-Leader', '001-Control Number',
       '008-Fixed-Length Data Elements-General Information', '245$a-Title',
       '245$b-Remainder of title', '245$f-Inclusive dates', '245$n',
       '245$p-Name of part/section of work',
       '260$a-Place of publication, distribution, etc.',
       '260$b-Name of publisher, distributor, etc.',
       '260$c-Date of publication',
       '264$a-Place of production, publication, distribution, manufacture',
       '264$b-Name of producer, publisher, distributor, manufacturer',
       '264$c-Date of production, publication, or distribution', '773$w',
       'Record Creation Date', 'Publication Status', 'Date 1', 'Date 2',
       'Place of Publication', 'Language', 'Modified Record', 'Bibliography',
       '6th', '8th'],
      dtype='object')

# Remove 'a', 'm', 's', 'b' to focus on outliars (Bibliography Column)

In [10]:
# Step 6: Filter rows based on 'Bibliography' values
exclude_chars = ['a', 'm', 's', 'b']
filtered_df = df_combined[~df_combined['Bibliography'].isin(exclude_chars)]

# Step 7: Select specific columns and add new columns from '000-Leader'
result_df = filtered_df[['000-Leader', 'Bibliography','6th','8th']].copy()
#result_df['6th'] = result_df['000-Leader'].str[5]  # 6th character (index starts from 0)
#result_df['8th'] = result_df['000-Leader'].str[7]  # 8th character

In [11]:
# Step 8: Display the filtered DataFrame and unique values in the '8th' column
result_df

,000-Leader,Bibliography,6th,8th
11,01231ntm a2200373 u 4500,t,n,m
25,01422ntm a2200373 u 4500,t,n,m
35,01076ntm a2200277 u 4500,t,n,m
56,01135ntm a2200325 u 4500,t,n,m
110,01441ntm a2200397 u 4500,t,n,m
...,...,...,...,...
134832,01196nem a2200313 u 4500,e,n,m
134841,01135nem a2200337 u 4500,e,n,m
134862,01165nem a2200301 u 4500,e,n,m
134935,01205ntm a2200301 u 4500,t,n,m


# Bibiographic outliar stats

The amount of outliars in the 7th position (bibiography) is equal to the amount of values in the 8th position.
The 8th position has values of "a", "b","m"

In [12]:
# Distinct and count values for '6th' column
distinct_values_6th = result_df['6th'].dropna().unique()
value_counts_6th = result_df['6th'].value_counts()
print("\nDistinct values in the 6th character column:")
for value in distinct_values_6th:
    print(f"- {value}")
print("\nCount of each distinct value in the 6th character column:")
print(tabulate(value_counts_6th.reset_index().values, headers=['Value', 'Count'], tablefmt='pretty'))

# Distinct and count values for 'Bibliography' column
distinct_values_bibliography = result_df['Bibliography'].dropna().unique()
value_counts_bibliography = result_df['Bibliography'].value_counts()
print("\nDistinct values in the Bibliography column:")
for value in distinct_values_bibliography:
    print(f"- {value}")
print("\nCount of each distinct value in the Bibliography column:")
print(tabulate(value_counts_bibliography.reset_index().values, headers=['Value', 'Count'], tablefmt='pretty'))

# Distinct and count values for '8th' column
distinct_values_8th = result_df['8th'].dropna().unique()
value_counts_8th = result_df['8th'].value_counts()
print("\nDistinct values in the 8th character column:")
for value in distinct_values_8th:
    print(f"- {value}")
print("\nCount of each distinct value in the 8th character column:")
print(tabulate(value_counts_8th.reset_index().values, headers=['Value', 'Count'], tablefmt='pretty'))

# Check if total count of Bibliography matches total count of 8th column
bibliography_total_count = value_counts_bibliography.sum()
eighth_total_count = value_counts_8th.sum()
print(f"\nTotal count in Bibliography column: {bibliography_total_count}")
print(f"Total count in 8th character column: {eighth_total_count}")
if bibliography_total_count == eighth_total_count:
    print("The counts match.")
else:
    print("The counts do not match.")


Distinct values in the 6th character column:
- n

Count of each distinct value in the 6th character column:
+-------+-------+
| Value | Count |
+-------+-------+
|   n   | 7401  |
+-------+-------+

Distinct values in the Bibliography column:
- t
- k
- e
- i
- g
- d

Count of each distinct value in the Bibliography column:
+-------+-------+
| Value | Count |
+-------+-------+
|   t   | 5619  |
|   e   | 1209  |
|   k   |  539  |
|   i   |  21   |
|   g   |  12   |
|   d   |   1   |
+-------+-------+

Distinct values in the 8th character column:
- m
- b
- a

Count of each distinct value in the 8th character column:
+-------+-------+
| Value | Count |
+-------+-------+
|   m   | 7381  |
|   b   |  19   |
|   a   |   1   |
+-------+-------+

Total count in Bibliography column: 7401
Total count in 8th character column: 7401
The counts match.


# Export csv of outliars in the Bibliography column

In [22]:
# Step 9: Export filtered rows to a CSV file with '6th' and '8th' columns from result_df, including '001-Control Number'
output_csv_path = '/content/filtered_bibliography.csv'
filtered_export_df = filtered_df.merge(result_df[['000-Leader', '6th', '8th']], on='000-Leader', how='left')

# Ensure only the columns that exist are included in the export
available_columns = [col for col in desired_columns if col in filtered_export_df.columns]
filtered_export_df = filtered_export_df[available_columns]

# Export to CSV
filtered_export_df.to_csv(output_csv_path, index=False)
print(f"\nFiltered rows exported to {output_csv_path}")


NameError: name 'desired_columns' is not defined

# Stats on 008-Fixed-Length Data Elements-

In [15]:
# Step 1: Filter to only include specific columns
df_combined_filtered = df_combined[[
    '008-Fixed-Length Data Elements-General Information',
    'Record Creation Date',
    'Publication Status',
    'Date 1',
    'Date 2',
    'Place of Publication',
    'Language',
    'Modified Record'
]].copy()

# Step 2: Display the filtered DataFrame
#.head()

Valid values for publication status according to MARC:

'b', 'c', 'd', 'i', 'k', 'm', 'n'



S,U,E,R,T,Q are used by Family Search

For langauge: gw may be an error


245a and b? ask Dali

Remove valid rows to focus on incorrect values

In [16]:
# Step 6: Create a new DataFrame that excludes specific values in 'Publication Status'
exclude_values = ['b', 'c', 'd', 'i', 'k', 'm', 'n']
df_combined_filtered = df_combined_filtered[~df_combined_filtered['Publication Status'].isin(exclude_values)].copy()

# Step 8: Show count of distinct values for 'Publication Status'
value_counts_publication_status = df_combined_filtered['Publication Status'].value_counts()
print("\nCount of each distinct value in the 'Publication Status' column:")
print(tabulate(value_counts_publication_status.reset_index().values, headers=['Publication Status', 'Count'], tablefmt='pretty'))


Count of each distinct value in the 'Publication Status' column:
+--------------------+--------+
| Publication Status | Count  |
+--------------------+--------+
|         s          | 102640 |
|         u          |  3480  |
|         e          |   54   |
|         r          |   19   |
|         t          |   6    |
|         q          |   2    |
+--------------------+--------+


In [18]:
# Step : Show count of distinct values for 'Publication Status' and 'Language'
# valid values are ++-
value_counts_publication_status = df_combined_filtered['Publication Status'].value_counts()
print("\nCount of each distinct value in the 'Publication Status' column:")
print(tabulate(value_counts_publication_status.reset_index().values, headers=['Publication Status', 'Count'], tablefmt='pretty'))

value_counts_language = df_combined_filtered['Language'].value_counts()
print("\nCount of each distinct value in the 'Language' column:")
print(tabulate(value_counts_language.reset_index().values, headers=['Language', 'Count'], tablefmt='pretty'))
# Step : Create a new DataFrame that excludes specific values in 'Publication Status'
exclude_values = ['b', 'c', 'd', 'i', 'k', 'm', 'n']
df_combined_filtered = df_combined_filtered[~df_combined_filtered['Publication Status'].isin(exclude_values)].copy()


Count of each distinct value in the 'Publication Status' column:
+--------------------+--------+
| Publication Status | Count  |
+--------------------+--------+
|         s          | 102640 |
|         u          |  3480  |
|         e          |   54   |
|         r          |   19   |
|         t          |   6    |
|         q          |   2    |
+--------------------+--------+

Count of each distinct value in the 'Language' column:
+----------+-------+
| Language | Count |
+----------+-------+
|   eng    | 62882 |
|   fre    | 13335 |
|   ger    | 8258  |
|   spa    | 6097  |
|   dut    | 4582  |
|   mul    | 4292  |
|   hun    | 1183  |
|   dan    | 1088  |
|   swe    | 1083  |
|   por    |  839  |
|   nor    |  625  |
|   ita    |  339  |
|   lat    |  311  |
|   ton    |  224  |
|   fin    |  192  |
|   pol    |  167  |
|   ind    |  159  |
|   cze    |  136  |
|   afr    |  61   |
|   srp    |  40   |
|   chi    |  35   |
|   rus    |  35   |
|   ice    |  27   |
|   slo    |

In [19]:
df['773$w'].unique()


array([     nan,   66442., 3274222.,  493092.,  752249.,  110054.,
         62970.,  724445.,  197244., 1829368.,  110004.])

In [20]:
df['773$w'].count()
#look at child periocials

28

**28 rows have a value in the 773$w column. Those rows all have 'b' as the 8th character in 000-Leader column. For the 7th letter ('Bibliographical info) all have the value 'a' with one exception being the value 't'**

**773$w values don't seem to correlate with values in column 001**

In [21]:
# Step 1: Filter rows where '773$w' is not null
filtered_df = df_combined[df_combined['773$w'].notna()]

# Step 2: Select only the desired columns
filtered_columns_df = filtered_df[['000-Leader', '001-Control Number', '773$w','Bibliography','8th']]

# Display the filtered DataFrame
filtered_columns_df


,000-Leader,001-Control Number,773$w,Bibliography,8th
45036,01344nab a2200361 u 4500,111914,66442.0,a,b
50663,01033nab a2200301 u 4500,125564,3274222.0,a,b
50906,02026nab a2200433 u 4500,126090,493092.0,a,b
50918,02246nab a2200445 u 4500,126145,493092.0,a,b
51843,01665nab a2200373 u 4500,128375,493092.0,a,b
52423,02591nab a2200481 u 4500,130037,493092.0,a,b
54456,01859nab a2200421 u 4500,134930,752249.0,a,b
57587,02034nab a2200397 u 4500,142496,110054.0,a,b
61507,01648ntb a2200421 u 4500,152065,62970.0,t,b
63156,02278nab a2200445 u 4500,156449,724445.0,a,b


Connect child records to parent records.

In [27]:
# Step: Filter out NA values from '773$w', convert to integers, and get the list of values
values_in_773w = df_combined['773$w'].dropna().astype(int).tolist()

# Display the list of values
values_in_773w



[66442,
 3274222,
 493092,
 493092,
 493092,
 493092,
 752249,
 110054,
 62970,
 724445,
 197244,
 1829368,
 110004,
 110004,
 110004,
 110004,
 110004,
 110004,
 110004,
 110004,
 110004,
 110004,
 110004,
 110004,
 110004,
 110004,
 110004,
 110004]

In [28]:
# Step: Filter rows in df_combined where '001-Control Number' matches values in values_in_773w
filtered_df = df_combined[df_combined['001-Control Number'].isin(values_in_773w)]

# Display the filtered DataFrame
filtered_df


,000-Leader,001-Control Number,008-Fixed-Length Data Elements-General Information,245$a-Title,245$b-Remainder of title,245$f-Inclusive dates,245$n,245$p-Name of part/section of work,"260$a-Place of publication, distribution, etc.","260$b-Name of publisher, distributor, etc.",...,Record Creation Date,Publication Status,Date 1,Date 2,Place of Publication,Language,Modified Record,Bibliography,6th,8th
25153,59195nam a2205917 u 4500,62970,841108m19341949 ||||| |||||||||||eng d,Genealogical reports,NaN,NaN,NaN,NaN,NaN,NaN,...,841108,m,1934,1949,,eng,,a,n,m
26673,02651nam a2200613 u 4500,66442,820115s1910 ||||| |||||||||||eng d,"Kent parish registers, marriages /",NaN,NaN,NaN,NaN,"London, England :","Phillimore & Co. Ltd.,",...,820115,s,1910,,,eng,,a,n,m
44208,03616nam a2200685 u 4500,110004,820319m19041906 ||||| |||||||||||eng d,Warwickshire parish registers /,NaN,NaN,NaN,NaN,"London, England :","Phillimore & Co. Ltd.,",...,820319,m,1904,1906,,eng,,a,n,m
44227,04140nam a2200769 u 4500,110054,820319m19091914 ||||| |||||||||||eng d,"Cheshire parish registers, marriages /",NaN,NaN,NaN,NaN,"London, England :","Phillimore & Co. Ltd.,",...,820319,m,1909,1914,,eng,,a,n,m
79757,01529nas a2200433 u 4500,197244,801121uuuuuuuuu ||||| | eng d,Members' interests,NaN,NaN,NaN,NaN,"[Birmingham, England :",Birmingham & Midland Society for Genealogy & H...,...,801121,u,uuuu,uuuu,,eng,,a,n,s
